#### 간단한 안티 크롤링 회피 방법

headers를 통해 서버에 데이터를 요청하면서 웹브라우저에 대한 정보를 같이 전송하여 마치 웹브라우저를 통해 데이터를 요청하는 것처럼 보이게 합니다

#### 요청값 규칙에 따라 URL 변화시키기
데이터를 수집하려는 페이지의 URL값에서 start번호를 자유롭게 변형시키기 위해서 변수를 추가해줍니다.
- 기존 URL: "https://search.naver.com/search.naver?where=news&query=코알라"
- 변수를 추가한 URL: "https://search.naver.com/search.naver?where=news&query=코알라&start="+str(n)   --> n은 반복시키려는 기사번호(1, 11, ...)

#### 컨테이너 수집

```python
1 <ul class="type01">
2 <li id="sp_nws1">...</li>
3 <li id="sp_nws2">...</li>
4 <li id="sp_nws5">...</li>
5 <li id="sp_nws6">...</li>
6 ...(생략)
```

1. li#sp_nws1 로 선택하는 경우
    - 첫번째 기사부분의 컨테이너를 확인하면 "sp_nws1"이라는 id를 가지고 있기 때문에 li#sp_nws1이라는 선택자로 데이터를 선택할 수 있습니다. 
    - 태그의 id는 유일한 이름이기때문에 첫번째 기사 데이터에 대해서는 정상적으로 선택이 가능하지만 모든 기사를 선택할 수 없습니다.


2. ul.type01 > li 로 선택하는 경우
    - 기사를 모두 선택할 수 없기 때문에 상위 태그에서부터 상하관계로 구분해주는 방법(자식, > )을 사용하면 ul.type01 > li 와 같이 선택자를 만들 수 있습니다.
    - 이 경우 페이지 위에서 확인할 수 있는 모든 기사를 선택할 수 있습니다.
    - 가장 적합한 선택자입니다.

#### 기사별 데이터 수집

```python
1 <a class="_sp_each_url _sp_each_title" ...(생략)>
2 오스트레일리아 폭염, 목마른 코알라
3 </a>
```

1. a._sp_each_url._sp_each_title ; 2가지 클래스를 모두 가지고있는 a 태그
    - 제목 데이터 중 일부는 2가지 클래스를 모두 가지고있고, 일부는 1가지 클래스만 가지고 있는 경우가 있기 때문에 10개 기사의 제목이 모두 선택되지 않을 수 있습니다. 
    - 모든 데이터를 가리키고 있지 못하므로 좋은 선택자가 아닙니다.
    
    
2. a._sp_each_url ;"_sp_each_url" 클래스를 가지고있는 a 태그
    - 제목 외에도 "_sp_each_url" 클래스를 가지고있는 경우가 있기 때문에 기사 개수 이상의 데이터가 선택됩니다.
    - 원하는 데이터보다 더 많은 데이터를 가리키고 있으므로 좋은 선택자가 아닙니다.
    
    
3. a._sp_each_title ;"_sp_each_title" 클래스를 가지고 있는 a 태그
    - 기사 개수만큼의 제목데이터만 선택하고 있습니다.
    - 원하는 데이터를 정확히 가리키고 있으므로 적절한 선택자입니다.

In [1]:
# 소스코드 가져오고 파싱하기
import requests
from bs4 import BeautifulSoup

# 검색어 직접 입력
keyword = input('검색어를 입력하세요: ')

# 반복1: 기사번호를 변경시키면서 데이터 수집을 반복하기
# 1 ~ 100까지 10단위로 반복(1, 11, ..., 91)
for n in range(1,100,10):
    raw = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query="+keyword+"&start="+str(n), 
                       headers={'User-Agent' : 'Morzilla/5.0'})
    html = BeautifulSoup(raw.text, "html.parser")
    articles = html.select("ul.type01>li")

    # 반복2: 기사에 대해서 반복하면 세부 정보 수집하기
    # 리스트를 사용한 반복문으로 모든 기사에 대해서 제목/언론사 출력
    for ar in articles:
        title = ar.select_one("a._sp_each_title").text
        source = ar.select_one("span._sp_each_source").text
        print(title, source)

검색어를 입력하세요: 코로나
文대통령 "코로나 이겨내고 위축된 경제 되살릴 수 있어"[전문] 파이낸셜뉴스언론사 선정
신천지 "오늘 이만희 총회장 코로나 진단검사"(종합) 연합뉴스언론사 선정
경북 경산서 생후 45일 아기 코로나19 확진…전국 최연소(종합) 연합뉴스언론사 선정
여야 "코로나 추경 2월 국회 내 처리…선거구획정 내일 마무리" SBS언론사 선정
청하 측 "코로나 19 음성 판정, 자가격리 진행 중"[공식] 스포츠서울
[속보]‘코로나19’ 18번째 사망자 발생… 대구 83세 남성 경향신문언론사 선정
[속보] 코로나 환자 210명 추가 확진...3736명으로 늘어 중앙일보
청하 스태프 2명 코로나19 확진…청하는 '음성'[전문] 이데일리
이만희 코로나19 검사 예정…신천지 "몰아세우지 말라"(종합) 노컷뉴스언론사 선정
코로나19 환자 376명 늘어 총 3526명…TK 3000명↑(종합) 뉴시스언론사 선정
코로나19 대구 환자, 광주서 치료한다…"달빛동맹"(종합) 연합뉴스언론사 선정
국내 코로나19 18번째 사망자 발생…83세 대구 남성(종합) 연합뉴스언론사 선정
청하 스태프 2명 코로나19 확진…청하는 '음성'[전문] 이데일리
[단독]이만희 신천지 총회장, 코로나 19 검사 받는다 채널A언론사 선정
대통령 "3·1정신으로 코로나 이겨야…TK 외롭지 않아" 미디어오늘언론사 선정
코로나 해외여행 취소 '대란'…위약금 분쟁 3배로 MBC언론사 선정
[단독] 황치열, 코로나19 성금 5천만원 기부…따뜻한 선행 엑스포츠뉴스
[속보]코로나19 확진자, 오늘 586명 추가···총 3736명 경향신문
'코로나19 급증' 천안서 10대 확진자 속출…2차 감염 본격 시작(종합) 연합뉴스언론사 선정
현대시티몰 가든파이브점, 코로나 19 확진자 방문에 임시 휴점 조선비즈언론사 선정
코로나19 탓 '최소화'한 3·1절 기념식…훈장수여도 생략(종합) 연합뉴스
'코로나19'에도 수출 15개월만에 장기하락 고리 끊었다 파이낸셜뉴스언론사 선정
생후 45일 남아 코로나 최연소 확진··